In [233]:
cd hackathon_baseline/

[Errno 2] No such file or directory: 'hackathon_baseline/'
/workspace/asquarerock-hackathon/hackathon_baseline


In [234]:
pip install -e .

Obtaining file:///workspace/asquarerock-hackathon/hackathon_baseline
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ge-hack
    Found existing installation: ge-hack 0.1
    Uninstalling ge-hack-0.1:
      Successfully uninstalled ge-hack-0.1
  Running setup.py develop for ge-hack
Note: you may need to restart the kernel to use updated packages.


In [235]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings(action='ignore')

import matplotlib.pyplot as plt

import copy as cp
import pickle as pkl
from importlib import reload

from sklearn.preprocessing import MinMaxScaler
### Metrics
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error as mse
from scipy.stats import spearmanr

import torch
import random
import re

# Models
from sklearn.linear_model import LinearRegression
import ge_hack.utils as utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [236]:
data_prefix_path = '/Data_hackathon/'
# Adapt this path based on where you have stored the data.
data_directory = data_prefix_path+'ccle/'

print(data_directory)

/Data_hackathon/ccle/


In [237]:
# List of the availables files
files = os.listdir(data_directory)
print(files)

['CCLE_mutations.csv', 'common_depmap', 'DepIoGenes_to_use.csv', 'sample_info.csv', 'DepIoGenes.csv', 'processed', 'CCLE_expression_full.csv', 'CCLE_expression.csv', 'gene_dependencies_training.csv', 'suppl_data', 'fingerprint_T.csv']


In [238]:
# Load all of the files you are interested in.
dict_data = {}

# Can modify next lines if you want a subset
df_to_load = ['sample_info.csv','fingerprint_T.csv',
              'DepIoGenes_to_use.csv', 'DepIoGenes.csv'
             ]

pickles_to_load = ['gene_lists/genes_expr_to_keep', 'gene_lists/genes_mut_to_keep']

files_to_load = df_to_load + pickles_to_load

utils.load_data_in_dict_and_print(data_directory,
                                  df_to_load,
                                  dict_data)

utils.load_data_in_dict_and_print(data_prefix_path,
                                  pickles_to_load,
                                  dict_data)
        

sample_info.csv
(1840, 28)
fingerprint_T.csv
(20597, 3115)
DepIoGenes_to_use.csv
(1227, 3)
DepIoGenes.csv
(1298, 3)
gene_lists/genes_expr_to_keep
6135
gene_lists/genes_mut_to_keep
4839


In [ ]:
# Can modify next lines if you want a subset
df_to_load = ["ccle_expression_training.csv", "gene_dependencies_training.csv",
             "ccle_expression_test.csv", "ccle_mutations_training.csv", "ccle_mutations_test.csv"] # "ccle_mutation_processed.csv",

# Load directly with index as DepMap id.
utils.load_data_in_dict_and_print(data_directory+"processed/",
                                  df_to_load,
                                  dict_data,
                                  index=True)

ccle_expression_training.csv
(804, 6135)
gene_dependencies_training.csv


In [ ]:
dep = dict_data["gene_dependencies_training.csv"]
dep = dep.set_index("DepMap_ID")
dep.head()

In [ ]:
# Merge training and test for ease of preprocessing.
# /!\ For a clean evaluation you should do preprocessing of train/test separately.
exp_prepro = pd.concat([dict_data["ccle_expression_training.csv"],dict_data["ccle_expression_test.csv"]])
# dict_data["ccle_expression_training.csv"]
exp_prepro

In [12]:
# Merge training and test for ease of preprocessing.
# /!\ For a clean evaluation you should do preprocessing of train/test separately.
mut_prepro = pd.concat([dict_data["ccle_mutations_training.csv"],dict_data["ccle_mutations_test.csv"]], axis=1).T
# dict_data["ccle_expression_training.csv"]
mut_prepro

Hugo_Symbol,A1CF,A2M,A2ML1,AADACL2,AARS2,AASDH,AASS,ABCA1,ABCA10,ABCA12,...,ZSCAN18,ZSCAN20,ZSCAN4,ZSCAN5B,ZSWIM2,ZSWIM4,ZSWIM5,ZSWIM8,ZZEF1,ZZZ3
ACH-000879,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000648,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ACH-000350,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ACH-000324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
ACH-000704,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACH-000860,1,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
ACH-000316,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
# Directly read depIo for which the selection was done in deepdep.
depio_full = dict_data['DepIoGenes.csv']
depio = dict_data['DepIoGenes_to_use.csv']
print(depio.shape)
depio.head()

(1227, 3)


,Std in original dep score across 278 CCLs,Cancer Gene Census,61 highly variable DepOIs
Gene dependency,,,
AARS2,221,FAUX,0
AASDHPPT,201,FAUX,0
ABCE1,211,FAUX,0
ABCG1,205,FAUX,0
ABHD11,211,FAUX,0


In [15]:
# We will focus on these ones as targets for the hackathon.
new_depIO = depio.loc[(depio['61 highly variable DepOIs']==1) & depio.index.isin(list(dep.columns))].index.values

fin = dict_data['fingerprint_T.csv']
print(fin.shape)
fingerprints_depIO = fin.loc[new_depIO]


(20597, 3115)


(59, 3115)

In [17]:
depedencies_depio = dep[new_depIO]
exp_prepro.shape, fingerprints_depIO.shape, depedencies_depio.shape, mut_prepro.shape

((1004, 6135), (59, 3115), (804, 59), (1004, 4839))

### get clinicals

In [35]:
patient_data = pd.read_csv('/Data_hackathon/Data_hackathon/ccle/sample_info.csv')
clinical = patient_data.loc[:,['primary_disease','primary_or_metastasis','lineage', "DepMap_ID"]]
clinical = clinical.set_index('DepMap_ID')
pd.get_dummies(clinical).to_csv('dummy_clinical.csv')

,primary_disease,primary_or_metastasis,lineage
DepMap_ID,,,
ACH-000016,Kidney Cancer,Metastasis,kidney
ACH-000032,Leukemia,NaN,blood
ACH-000033,Lung Cancer,Metastasis,lung
ACH-000043,Non-Cancerous,Metastasis,fibroblast
ACH-000049,Non-Cancerous,NaN,kidney
...,...,...,...
ACH-002393,Lymphoma,Metastasis,lymphocyte
ACH-002394,Colon/Colorectal Cancer,NaN,colorectal
ACH-002395,Liver Cancer,NaN,liver


### AE preprocessing

In [19]:
# Scaling?
scaler = MinMaxScaler()
exp_prepro_scaled = scaler.fit_transform(exp_prepro)
exp_prepro_scaled = pd.DataFrame(exp_prepro_scaled,
                                 index = exp_prepro.index)


In [44]:
exp_prepro.hist()

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f8981457790> (for post_execute):



KeyboardInterrupt



In [20]:
keep_genes = list(exp_prepro.columns)

keep_genes.sort()

print(len(keep_genes))

exp_prepro = exp_prepro[keep_genes]

6135


In [22]:
from ge_hack.models.autoencoder import AutoEncoder

autoencoder_rna = AutoEncoder(in_features=exp_prepro.shape[1],
                    repr_dim=300,
                    hidden=[1000],
                    bias=False,
                    num_epochs=400,
                    dropout=False,
                    batch_size=512,
                    learning_rate=1e-4,
                    linear=False,
                    device="cpu"
                             )

autoencoder_fgps = AutoEncoder(in_features=fingerprints_depIO.shape[1],
                    repr_dim=50,
                    hidden=[1000,100],
                    bias=False,
                    num_epochs=100,
                    dropout=0.,
                    batch_size=64,
                    learning_rate=1e-3,
                    linear=False,
                    device="cuda:0"
                              )

autoencoder_mut = AutoEncoder(
    in_features=mut_prepro.shape[1],
    repr_dim=50,
    hidden=[1000, 500, 100],
    bias=True,
    num_epochs=100,
    batch_size=2048,
    learning_rate=1e-5,
    linear=False,
    device='cuda:0',
    dropout=0.
)

In [25]:
autoencoder_rna.load_state_dict(torch.load("ge_hack/saved_models/autoencoder_rna.pth"))
autoencoder_fgps.load_state_dict(torch.load("ge_hack/saved_models/autoencoder_fgps.pth"))
autoencoder_mut.load_state_dict(torch.load("ge_hack/saved_models/autoencoder_mut.pth"))

<All keys matched successfully>

In [48]:
autoencoder_rna.eval()
autoencoder_mut.eval()
autoencoder_fgps.eval()


with torch.no_grad():
    z_exp = autoencoder_rna.transform(exp_prepro.values)
    exp_prepro_reduced = pd.DataFrame(z_exp,
                                      index=exp_prepro.index
                                    )
    z_fgps = autoencoder_fgps.transform(fingerprints_depIO.values)
    fingerprints_depIO_reduced = pd.DataFrame(z_fgps,
                                      index=fingerprints_depIO.index
                                    )
    z_mut = autoencoder_mut.transform(mut_prepro.values)
    mut_prepro_reduced = pd.DataFrame(z_mut,
                                      index=mut_prepro.index
                                    )

100%|██████████| 1/1 [00:00<00:00, 43.83it/s]


In [81]:
cellline_data = pd.concat([exp_prepro_reduced, mut_prepro_reduced], axis=1)

In [ ]:
, clinical, how="inner", left_index=True, right_index=True)

In [73]:
cellline_data.shape

(1004, 353)

## making the multi-index stuff

In [82]:
s1 = cellline_data.shape
s2 = fingerprints_depIO_reduced.shape


s1, s2, s3

((1004, 350), (59, 50), (1004, 50))

In [83]:
arrays_of_index = [list(np.repeat(cellline_data.index,s2[0])), list(fingerprints_depIO_reduced.index)*s1[0]]

tuples = list(zip(*arrays_of_index))

cl_depio_index = pd.MultiIndex.from_tuples(tuples, names=["DepMap_id", "DepIO"])

In [84]:
len(tuples)

59236

In [85]:
# Create matrix to do numpy operations.

B = np.zeros((s2[0]*s1[0],s1[0]))

Ep = np.zeros((s1[0],s1[1]+s2[1]))

Ep[0:s1[0], 0:s1[1]] = cellline_data.values

ones_s20 = np.ones((s2[0],1))

# Fill B with ones_f0 vector.
for i in range(s1[0]):
    B[0+s2[0]*i:s2[0]*(i+1),i:i+1] = ones_s20

# Add all the ei to the C.
C = np.dot(B,Ep)

In [86]:
for i in range(s1[0]):
    # Block size s2[0]x s2[1] filled by s2 matrix
    C[0+i*s2[0]:s2[0]+i*s2[0],  s1[1]:]=fingerprints_depIO_reduced.values

In [87]:
# Add prefix to columns to identify both
exp_prepro_reduced.columns = ["exp_" + str(i) for i in exp_prepro_reduced.columns]
fingerprints_depIO_reduced.columns = ["fpgs_" + str(i) for i in fingerprints_depIO_reduced.columns]
mut_prepro_reduced.columns = ["mut_" + str(i) for i in mut_prepro_reduced.columns]


In [88]:
concatenated_matrix = pd.DataFrame(data=C,
                                   columns =list(exp_prepro_reduced.columns)+list(fingerprints_depIO_reduced.columns)+list(mut_prepro_reduced.columns), index = cl_depio_index)

In [89]:
concatenated_matrix.shape

(59236, 400)

In [91]:
concatenated_matrix.reset_index()

,DepMap_id,DepIO,exp_exp_0,exp_exp_1,exp_exp_2,exp_exp_3,exp_exp_4,exp_exp_5,exp_exp_6,exp_exp_7,...,mut_40,mut_41,mut_42,mut_43,mut_44,mut_45,mut_46,mut_47,mut_48,mut_49
0,ACH-001289,ACO2,0.0,4.262052,3.138903,5.946784,0.000000,12.363298,0.0,5.530138,...,0.0,1.467920,1.282697,0.0,0.000000,2.830050,0.000000,0.000000,2.080016,1.821016
1,ACH-001289,ADSL,0.0,4.262052,3.138903,5.946784,0.000000,12.363298,0.0,5.530138,...,0.0,0.000000,0.000000,0.0,0.000000,0.350823,0.518096,0.000000,0.933215,0.135900
2,ACH-001289,AMBRA1,0.0,4.262052,3.138903,5.946784,0.000000,12.363298,0.0,5.530138,...,0.0,1.641030,1.576812,0.0,0.773569,2.674485,0.944132,0.000000,0.544670,2.438804
3,ACH-001289,ATP1A1,0.0,4.262052,3.138903,5.946784,0.000000,12.363298,0.0,5.530138,...,0.0,0.444896,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.492978,2.529439
4,ACH-001289,BCL2L1,0.0,4.262052,3.138903,5.946784,0.000000,12.363298,0.0,5.530138,...,0.0,0.432330,0.006156,0.0,1.220592,0.000000,1.057029,0.949027,4.012289,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59231,ACH-000200,TSC2,0.0,8.558276,4.917293,6.980165,0.687493,13.586962,0.0,6.134007,...,0.0,1.269587,0.056395,0.0,0.644132,2.219785,0.872884,0.000000,0.781441,2.132617
59232,ACH-000200,TXNRD1,0.0,8.558276,4.917293,6.980165,0.687493,13.586962,0.0,6.134007,...,0.0,2.173475,4.447536,0.0,6.107875,3.798199,1.543986,0.000000,0.463834,2.291834
59233,ACH-000200,TYMS,0.0,8.558276,4.917293,6.980165,0.687493,13.586962,0.0,6.134007,...,0.0,0.000000,0.000000,0.0,0.000000,0.597987,0.000000,0.000000,1.378530,0.000000
59234,ACH-000200,WDR1,0.0,8.558276,4.917293,6.980165,0.687493,13.586962,0.0,6.134007,...,0.0,1.974134,0.880746,0.0,2.290071,1.160875,1.030722,0.000000,1.614219,2.676561


In [163]:
X = pd.merge(concatenated_matrix.reset_index(),clinical, left_on=['DepMap_id'], right_index = True).iloc[:,np.r_[0:402,404]]

In [108]:
y_cl_depio = pd.DataFrame(0,
                          index=cl_depio_index,
                          columns=["Gene dependency"])

In [109]:
for i in depedencies_depio.index:
    y_cl_depio.loc[i,"Gene dependency"] = depedencies_depio.loc[i].values

In [165]:
X.set_index(['DepMap_id','DepIO'], inplace=True)
y = y_cl_depio

In [166]:
X

exp_exp_0  exp_exp_1  exp_exp_2  exp_exp_3  exp_exp_4  \
DepMap_id  DepIO                                                           
ACH-001289 ACO2          0.0   4.262052   3.138903   5.946784   0.000000   
           ADSL          0.0   4.262052   3.138903   5.946784   0.000000   
           AMBRA1        0.0   4.262052   3.138903   5.946784   0.000000   
           ATP1A1        0.0   4.262052   3.138903   5.946784   0.000000   
           BCL2L1        0.0   4.262052   3.138903   5.946784   0.000000   
...                      ...        ...        ...        ...        ...   
ACH-000200 TSC2          0.0   8.558276   4.917293   6.980165   0.687493   
           TXNRD1        0.0   8.558276   4.917293   6.980165   0.687493   
           TYMS          0.0   8.558276   4.917293   6.980165   0.687493   
           WDR1          0.0   8.558276   4.917293   6.980165   0.687493   
           YRDC          0.0   8.558276   4.917293   6.980165   0.687493   

                   exp_exp_5  exp_exp_6  exp_exp_7  exp_exp_8  exp_exp_9  ...  \
DepMap_id  DepIO                                                          ...   
ACH-001289 ACO2    12.363298        0.0   5.530138   4.269902   2.189084  ...   
           ADSL    12.363298        0.0   5.530138   4.269902   2.189084  ...   
           AMBRA1  12.363298        0.0   5.530138   4.269902   2.189084  ...   
           ATP1A1  12.363298        0.0   5.530138   4.269902   2.189084  ...   
           BCL2L1  12.363298        0.0   5.530138   4.269902   2.189084  ...   
...                      ...        ...        ...        ...        ...  ...   
ACH-000200 TSC2    13.586962        0.0   6.134007  12.987433   3.281682  ...   
           TXNRD1  13.586962        0.0   6.134007  12.987433   3.281682  ...   
           TYMS    13.586962        0.0   6.134007  12.987433   3.281682  ...   
           WDR1    13.586962        0.0   6.134007  12.987433   3.281682  ...   
           YRDC    13.586962        0.0   6.134007  12.987433   3.281682  ...   

                     mut_41    mut_42  mut_43    mut_44    mut_45    mut_46  \
DepMap_id  DepIO                                                              
ACH-001289 ACO2    1.467920  1.282697     0.0  0.000000  2.830050  0.000000   
           ADSL    0.000000  0.000000     0.0  0.000000  0.350823  0.518096   
           AMBRA1  1.641030  1.576812     0.0  0.773569  2.674485  0.944132   
           ATP1A1  0.444896  0.000000     0.0  0.000000  0.000000  0.000000   
           BCL2L1  0.432330  0.006156     0.0  1.220592  0.000000  1.057029   
...                     ...       ...     ...       ...       ...       ...   
ACH-000200 TSC2    1.269587  0.056395     0.0  0.644132  2.219785  0.872884   
           TXNRD1  2.173475  4.447536     0.0  6.107875  3.798199  1.543986   
           TYMS    0.000000  0.000000     0.0  0.000000  0.597987  0.000000   
           WDR1    1.974134  0.880746     0.0  2.290071  1.160875  1.030722   
           YRDC    0.000000  1.615652     0.0  1.867827  1.517505  0.000000   

                     mut_47    mut_48    mut_49                 lineage  
DepMap_id  DepIO                                                         
ACH-001289 ACO2    0.000000  2.080016  1.821016             soft_tissue  
           ADSL    0.000000  0.933215  0.135900             soft_tissue  
           AMBRA1  0.000000  0.544670  2.438804             soft_tissue  
           ATP1A1  0.000000  1.492978  2.529439             soft_tissue  
           BCL2L1  0.949027  4.012289  0.000000             soft_tissue  
...                     ...       ...       ...                     ...  
ACH-000200 TSC2    0.000000  0.781441  2.132617  central_nervous_system  
           TXNRD1  0.000000  0.463834  2.291834  central_nervous_system  
           TYMS    0.000000  1.378530  0.000000  central_nervous_system  
           WDR1    0.000000  1.614219  2.676561  central_nervous_system  
           YRDC    0.000000  0.066948  0.147046  central_ner

### train test split

In [167]:
import pickle
# Load train test index.
with open(data_directory+"processed/"+"DepMapID_trainingset", "rb") as fp:   #Pickling
    train_ids = pickle.load(fp)
    
with open(data_directory+"processed/"+"DepMapID_testingset", "rb") as fp:   #Pickling
    test_ids = pickle.load(fp)

In [168]:
# Classic train test split for debugging purpose. (keep this one)
X_train, X_eval, y_train, y_eval = train_test_split(X.loc[train_ids],
                                                    y.loc[train_ids],
                                                    test_size=0.1,
                                                    random_state=42
                                                   )

X_train.shape, X_eval.shape, y_train.shape, y_eval.shape

((42692, 401), (4744, 401), (42692, 1), (4744, 1))

In [169]:
lineage_list = list(set(clinical.lineage))


In [171]:
X_train[X_train.lineage == lineage_list[0]].iloc[:,:-1]

,,exp_exp_0,exp_exp_1,exp_exp_2,exp_exp_3,exp_exp_4,exp_exp_5,exp_exp_6,exp_exp_7,exp_exp_8,exp_exp_9,...,mut_40,mut_41,mut_42,mut_43,mut_44,mut_45,mut_46,mut_47,mut_48,mut_49
DepMap_id,DepIO,,,,,,,,,,,,,,,,,,,,,
ACH-000272,MYC,0.0,6.747735,4.518071,7.582827,3.533693,13.824612,0.0,3.775694,10.848234,6.388714,...,0.000000,2.311120,0.595006,0.0,1.086494,0.433925,0.698704,0.0,0.000000,0.876427
ACH-000189,TAF5L,0.0,5.714624,4.685894,10.208841,5.680552,14.353985,0.0,5.958707,12.078744,9.094378,...,0.000000,0.000000,1.671195,0.0,0.851661,1.163194,0.914061,0.0,1.827186,0.774393
ACH-001688,C1orf109,0.0,6.072840,7.091409,6.824103,4.609385,10.869410,0.0,4.586490,10.135651,5.766654,...,0.000000,0.656482,0.000000,0.0,0.259145,0.000000,0.607775,0.0,1.281085,1.105686
ACH-000907,KEAP1,0.0,7.662282,7.264705,5.719409,5.085812,14.223945,0.0,5.952466,13.986370,7.362463,...,0.000000,0.789891,0.546225,0.0,0.473824,2.458611,0.870423,0.0,0.733340,2.087422
ACH-000246,TAF5L,0.0,7.262815,2.882369,8.147053,0.317189,13.117407,0.0,1.453843,9.705510,5.107198,...,0.000000,0.000000,1.671195,0.0,0.851661,1.163194,0.914061,0.0,1.827186,0.774393
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-000484,EP300,0.0,7.985190,5.436757,9.962814,2.525845,13.852516,0.0,3.631937,11.177383,7.252681,...,0.000000,0.000000,1.071688,0.0,1.867107,1.685472,0.344771,0.0,0.759640,0.114572
ACH-000495,SDHB,0.0,6.869037,5.165813,7.984434,5.621201,13.985111,0.0,5.006714,12.741401,8.362364,...,0.019784,0.956953,2.205054,0.0,0.000000,1.085567,0.708925,0.0,0.682810,3.718467
ACH-000385,TFRC,0.0,7.843802,4.143964,6.252289,3.751057,12.700430,0.0,6.445818,13.310784,5.226022,...,0.000000,1.279165,0.000000,0.0,1.149280,0.000000,0.205988,0.0,0.196499,1.182504


In [173]:
y_train[X_train.lineage == lineage_list[0]]

,,Gene dependency
DepMap_id,DepIO,
ACH-000272,MYC,0.838948
ACH-000189,TAF5L,0.000081
ACH-001688,C1orf109,0.910548
ACH-000907,KEAP1,0.000687
ACH-000246,TAF5L,0.000073
...,...,...
ACH-000484,EP300,0.593056
ACH-000495,SDHB,0.087884
ACH-000385,TFRC,0.611030


In [184]:
X.loc[test_ids].groupby('lineage').exp_exp_0.count()

lineage
bile_duct                     354
blood                         649
bone                          354
breast                        531
central_nervous_system        826
cervix                        177
colorectal                    649
esophagus                     354
eye                           118
gastric                       354
kidney                        354
liver                         236
lung                         1534
lymphocyte                    413
ovary                         649
pancreas                      531
peripheral_nervous_system     354
plasma_cell                   236
prostate                       59
skin                          767
soft_tissue                   649
thyroid                       177
unknown                       118
upper_aerodigestive           590
urinary_tract                 354
uterus                        413
Name: exp_exp_0, dtype: int64

In [185]:
X.loc[train_ids].groupby('lineage').exp_exp_0.count()

lineage
bile_duct                    1829
blood                        2714
bone                         1357
breast                       2065
central_nervous_system       3363
cervix                        767
colorectal                   2596
embryo                         59
epidermoid_carcinoma           59
esophagus                    1357
eye                           708
gastric                      1416
kidney                       1416
liver                        1062
lung                         6136
lymphocyte                   1711
ovary                        2537
pancreas                     2124
peripheral_nervous_system    1357
plasma_cell                   885
prostate                      413
skin                         3186
soft_tissue                  2124
thyroid                       590
unknown                        59
upper_aerodigestive          2478
urinary_tract                1475
uterus                       1593
Name: exp_exp_0, dtype: int64

In [ ]:
## there is less train data than test data from some diseases

In [186]:
%%time
import xgboost as xgb

params = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': .03,
         'objective': 'reg:pseudohubererror',
          }
models = {}
for lineage in lineage_list:
    X_lineage_spe =X_train[X_train.lineage == lineage].iloc[:,:-1]
    y_lineage = y_train[X_train.lineage == lineage]
    dtrain = xgb.DMatrix(X_lineage_spe)
    dtrain.set_label(y_lineage)
    models[lineage] = xgb.train(params, dtrain, 150)


[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41:10] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:41

[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42:17] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:42

CPU times: user 8min 47s, sys: 1.29 s, total: 8min 49s
Wall time: 2min 12s


In [196]:
y_pred_train = y_train.copy() * 0
y_pred_eval = y_eval.copy() * 0

In [201]:
for lineage in lineage_list:
    y_pred_train[X_train.lineage == lineage] = models[lineage].predict(xgb.DMatrix(X_train[X_train.lineage == lineage].iloc[:,:-1]))[:,None]
    y_pred_eval[X_eval.lineage == lineage] = models[lineage].predict(xgb.DMatrix(X_eval[X_eval.lineage == lineage].iloc[:,:-1]))[:,None]

[21:51:25] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:51:25] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:51:26] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:51:26] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0


In [ ]:
[model.predict(xgb.DMatrix(X_train[X_train.lineage == lineage].iloc[:,:-1])) for lineage, model in models.items()

In [188]:
y_pred_train = np.concat([model.predict(xgb.DMatrix(X_train[X_train.lineage == lineage].iloc[:,:-1])) for lineage, model in models.items()]).T
y_pred_eval = np.array([model.predict(xgb.DMatrix(X_eval[X_eval.lineage == lineage].iloc[:,:-1])) for lineage, model in models.items()]).T
y_pred_train.shape, y_pred_eval.shape

[21:43:12] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:43:13] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:43:14] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:43:14] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0


((30,), (30,))

In [202]:
y_pred_train = pd.DataFrame(y_pred_train.values, index = X_train.index, columns=["Predicted Gene dependency"])
y_pred_eval = pd.DataFrame(y_pred_eval.values, index = X_eval.index, columns=["Predicted Gene dependency"])


In [203]:
score = r2_score(y_train,y_pred_train)
print("R2 score on the train set: {}".format(score))

R2 score on the train set: 0.752989337420698


In [204]:
score = r2_score(y_eval,y_pred_eval)
print("R2 score on the test set: {}".format(score))

R2 score on the test set: 0.4896873048615369


In [205]:
score = pearsonr(y_train.values[:,0],y_pred_train.values.flatten())[0]
print("Pearson score on the train set: {}".format(score))

Pearson score on the train set: 0.8771187575464165


In [206]:
score = pearsonr(y_eval.values[:,0],y_pred_eval.values.flatten())[0]
print("Pearson score on the test set: {}".format(score))

Pearson score on the test set: 0.7002204912444784


In [207]:
score = mse(y_train.values[:,0],y_pred_train)
print("MSE score on the train set: {}".format(score))

MSE score on the train set: 0.03539882384510104


In [208]:
score = mse(y_eval.values[:,0],y_pred_eval)
print("MSE on the test set: {}".format(score))

MSE on the test set: 0.07281789828452592


In [209]:
score = spearmanr(y_train.values[:,0],y_pred_train.values.flatten())[0]
print("Spearman score on the train set: {}".format(score))

Spearman score on the train set: 0.8780028199941076


In [210]:
score = spearmanr(y_eval.values[:,0],y_pred_eval.values.flatten())[0]
print("Spearman score on the test set: {}".format(score))

Spearman score on the test set: 0.7149311064446581


In [212]:
reload(utils);
metrics = ["r2","pearson", "mse","spearman"]

res = utils.evaluated_per_depio(y_eval,y_pred_eval,metric = "spearman")

#res = utils.evaluated_per_cell_line(y_eval,y_pred_eval,metric = "spearman")

In [213]:
res = utils.evaluated_per_depio(y_eval,y_pred_eval,metric = "spearman")

In [214]:
res.describe()

count    59.000000
mean      0.362930
std       0.176956
min      -0.351872
25%       0.243456
50%       0.393948
75%       0.479001
max       0.709842
dtype: float64

In [215]:
print("Per DepOI average correlation: mean {} +- std {}".format(res.describe()["mean"],
                                                                res.describe()["std"]))

Per DepOI average correlation: mean 0.3629302819750933 +- std 0.1769561593358462


In [216]:
y_train_full = y.loc[train_ids]
X_train_full = X.loc[train_ids]
X_test = X.loc[test_ids]

In [220]:
%%time
import xgboost as xgb

params = {'booster': 'dart',
         'max_depth': 5, 'learning_rate': .03,
         'objective': 'reg:pseudohubererror',
          }
models = {}
for lineage in lineage_list:
    X_lineage_spe =X_train_full[X_train_full.lineage == lineage].iloc[:,:-1]
    y_lineage = y_train_full[X_train_full.lineage == lineage]
    dtrain = xgb.DMatrix(X_lineage_spe)
    dtrain.set_label(y_lineage)
    models[lineage] = xgb.train(params, dtrain, 150)


[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55:55] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:55

[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57:06] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:57

CPU times: user 9min 29s, sys: 1.27 s, total: 9min 30s
Wall time: 2min 23s


In [221]:
y_pred_test_df = pd.DataFrame(0,index = y_cl_depio.loc[test_ids].index,columns=["Gene dependency"])

In [222]:
for lineage in lineage_list:
    y_pred_test_df[X.loc[test_ids].lineage == lineage] = models[lineage].predict(xgb.DMatrix(X.loc[test_ids][X.loc[test_ids].lineage == lineage].iloc[:,:-1]))[:,None]


[21:59:46] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:59:46] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:59:47] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0
[21:59:47] WARNING: ../src/learner.cc:1350: Empty dataset at worker: 0


In [223]:
y_pred_test_df

Gene dependency
DepMap_id  DepIO                  
ACH-000809 ACO2           0.783648
           ADSL           0.724210
           AMBRA1         0.369637
           ATP1A1         0.898645
           BCL2L1         0.717582
...                            ...
ACH-000120 TSC2           0.090696
           TXNRD1         0.756013
           TYMS           0.351006
           WDR1           0.678252
           YRDC           0.737313

[11800 rows x 1 columns]

In [224]:
y_pred_test_df['id'] = y_pred_test_df.index.map('-'.join)

In [225]:
"ACH-000900" in set(train_ids)

True

In [226]:
y_pred_test_df.head()

Gene dependency                 id
DepMap_id  DepIO                                     
ACH-000809 ACO2           0.783648    ACH-000809-ACO2
           ADSL           0.724210    ACH-000809-ADSL
           AMBRA1         0.369637  ACH-000809-AMBRA1
           ATP1A1         0.898645  ACH-000809-ATP1A1
           BCL2L1         0.717582  ACH-000809-BCL2L1

In [227]:
y_pred_test_df.set_index('id',inplace=True)

In [228]:
y_pred_test_df.to_csv("y_pred_test_with_on_models_per_lineage.csv")

In [230]:
# Check your csv
y_submission_test = pd.read_csv("y_pred_test_with_on_models_per_lineage.csv")

In [231]:
y_submission_test

,id,Gene dependency
0,ACH-000809-ACO2,0.783648
1,ACH-000809-ADSL,0.724210
2,ACH-000809-AMBRA1,0.369637
3,ACH-000809-ATP1A1,0.898645
4,ACH-000809-BCL2L1,0.717582
...,...,...
11795,ACH-000120-TSC2,0.090696
11796,ACH-000120-TXNRD1,0.756013
11797,ACH-000120-TYMS,0.351006
11798,ACH-000120-WDR1,0.678252


In [232]:
ls

dummy_clinical.csv  README.md
ge_hack/            requirements.txt
ge_hack.egg-info/   setup.py
images/             tools/
LICENSE             y_pred_test_with_on_models_per_lineage.csv


# Full deep models finetuned.